In [1]:
# Reading the data in makes sense to structure a little bit
import random

def read_data_one_lang(lang,part):
    """Reads one file for one language. Returns data in the form of pairs of (lang,line)"""
    filename="language-identification/{}_{}.txt".format(lang,part)
    result=[] #this will be the list of pairs (lang,line)
    with open(filename) as f:
        for line in f:
            line=line.strip()
            result.append((lang,line)) 
    return result


def read_data_all_langs(part):
    """Reads train, test or dev data for all languages. part can be train, test, or devel"""
    #glob
    data=[]
    for lang in ("en","es","et","fi","pt"):
        pairs=read_data_one_lang(lang,part)
        data.extend(pairs) #just add these lines to the end
    #...done
    #but now they come in the order of languages
    #we really must scramble these!
    random.shuffle(data)
    
    #let's yet separate the labels and lines, we will need that anyway
    labels=[label for label,line in data]
    lines=[line for label,line in data]
    return labels,lines

labels_train,lines_train=read_data_all_langs("train")
labels_dev,lines_dev=read_data_all_langs("devel")
for label,line in zip(labels_train[:5],lines_train[:5]):
    print(label,"   ",line[:30],"...")
#and beyond this point, exactly same code is applicable as before

et     Sest viimased on palju ilusama ...
fi     Tähänkin on varmasti jäsenvalt ...
en     Thanks. ...
es     Muchos deltanos han optado por ...
es     García Rojas escapó el 21 de f ...


In [2]:
from sklearn.feature_extraction.text import CountVectorizer

#1-3 character grams
vectorizer=CountVectorizer(max_features=100000,binary=True,ngram_range=(3,3),analyzer="char_wb")
feature_matrix_train=vectorizer.fit_transform(lines_train)
feature_matrix_dev=vectorizer.transform(lines_dev)

from sklearn.preprocessing import LabelEncoder

label_encoder=LabelEncoder() #Turns class labels into integers
class_numbers_train=label_encoder.fit_transform(labels_train)
class_numbers_dev=label_encoder.fit_transform(labels_dev)

print("class_numbers shape=",class_numbers_train.shape)
print("class labels",label_encoder.classes_) #this will let us translate back from indices to labels



class_numbers shape= (5000,)
class labels ['en' 'es' 'et' 'fi' 'pt']


The default version of TensorFlow in Colab will switch to TensorFlow 2.x on the 27th of March, 2020.
We recommend you upgrade now or ensure your notebook will continue to use TensorFlow 1.x via the %tensorflow_version 1.x magic: more info.

https://www.tensorflow.org/guide/migrate
https://colab.research.google.com/notebooks/tensorflow_version.ipynb


In [4]:
# !pip install tensorflow==1.14.0 # to run with old tf with which the code is made
# The default version of TensorFlow in Colab will switch to TensorFlow 2.x on the 27th of March, 2020.
# 
import keras 
from keras.models import Model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

example_count,feature_count=feature_matrix_train.shape
class_count=len(label_encoder.classes_)

inp=Input(shape=(feature_count,))
hidden=Dense(20,activation="tanh")(inp)
outp=Dense(class_count,activation="softmax")(hidden)
model=Model(inputs=[inp], outputs=[outp])

stop_cb=EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, mode='auto', baseline=None, restore_best_weights=True)

model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=['accuracy'])
hist=model.fit(feature_matrix_train,class_numbers_train,batch_size=100,verbose=1,epochs=25,validation_data=(feature_matrix_dev,class_numbers_dev),callbacks=[stop_cb])

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_st






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 5000 samples, validate on 5000 samples
Epoch 1/25
5000/5000 [==============================] - 1s 266us/step - loss: 0.6821 - acc: 0.9160 - val_loss: 0.3269 - val_acc: 0.9784
Epoch 2/25
1300/5000 [======>.......................] - ETA: 0s - loss: 0.2621 - acc: 0.9869

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


5000/5000 [==============================] - 1s 218us/step - loss: 0.2118 - acc: 0.9878 - val_loss: 0.2004 - val_acc: 0.9816
Epoch 3/25
5000/5000 [==============================] - 1s 214us/step - loss: 0.1257 - acc: 0.9928 - val_loss: 0.1505 - val_acc: 0.9822
Epoch 4/25
5000/5000 [==============================] - 1s 222us/step - loss: 0.0849 - acc: 0.9972 - val_loss: 0.1238 - val_acc: 0.9828
Epoch 5/25
5000/5000 [==============================] - 1s 217us/step - loss: 0.0610 - acc: 0.9982 - val_loss: 0.1076 - val_acc: 0.9830
Epoch 6/25
5000/5000 [==============================] - 1s 217us/step - loss: 0.0457 - acc: 0.9990 - val_loss: 0.0969 - val_acc: 0.9838
Epoch 7/25
5000/5000 [==============================] - 1s 219us/step - loss: 0.0352 - acc: 0.9992 - val_loss: 0.0894 - val_acc: 0.9832
Epoch 8/25
5000/5000 [==============================] - 1s 214us/step - loss: 0.0278 - acc: 0.9998 - val_loss: 0.0839 - val_acc: 0.9826
Epoch 9/25
5000/5000 [==============================] - 1s 

* Let's try to identify misclassified documents

In [5]:
import numpy

predictions=model.predict(feature_matrix_dev)
pred_classes=numpy.argmax(predictions,axis=-1)
for pred,correct,txt_line in zip(pred_classes,labels_dev,lines_dev):
    pred_label=label_encoder.classes_[pred]
    if pred_label!=correct:
        print("Prediction:",pred_label,"Correct:",correct,"Text:",txt_line)



Prediction: en Correct: pt Text: « Peppermint Tea House-- The Best of Shoukichi Kina».
Prediction: es Correct: pt Text: Suficientemente refrescante e estival.
Prediction: en Correct: et Text: Üle tuhande meedialogo
Prediction: en Correct: fi Text: I
Prediction: en Correct: pt Text: Em um único ano, 1937, eles compuseram« They Can't Take That Away From Me»,« Let's Call the Whole Thing Off»,« A Foggy Day»,« Nice Work if You Can Get it»,« They All Laughed»,« Love Walked In»' e« Love Is Here to Stay»', e essas são apenas as que ficaram universalmente conhecidas.
Prediction: pt Correct: es Text: Florece de marzo a mayo.
Prediction: en Correct: es Text: Apareció en un artículo de The Alternate View:" Boomerang and the Sound of the Big Bang"( January 2001).
Prediction: en Correct: et Text: Virginia- odavam järglane.
Prediction: et Correct: fi Text: Talovahtina
Prediction: en Correct: fi Text: * KTM Fahrrad GmbH(polkupyörät)
Prediction: en Correct: pt Text: Faber and Faber
Prediction: en Corre

In [6]:
data_in=vectorizer.transform(["sdfjfj fsdjfoj fsjofs fjskf fjsklf","I really think this should be classified as English"])
print(label_encoder.classes_)
model.predict(data_in)

['en' 'es' 'et' 'fi' 'pt']


array([[4.9812174e-01, 5.3042203e-02, 1.5040587e-01, 1.4734639e-01,
        1.5108387e-01],
       [9.9988163e-01, 5.4350170e-05, 1.6178319e-05, 5.2756500e-06,
        4.2504202e-05]], dtype=float32)